本案例介绍了如何构建一个基于Llama 2的聊天机器人，该机器人能够在浏览器上运行，并能够根据自己的数据回答问题。主要内容包括：

1. **部署Llama 2 7B**：使用文本生成推理框架将Llama 2 7B作为API服务器部署。
2. **构建聊天机器人**：使用Gradio构建聊天机器人，并连接到服务器。
3. **增加检索增强生成（RAG）功能**：基于入门指南，增加Llama 2特定知识的RAG能力。
4. **RAG架构**：RAG是Meta在2020年发明的一种流行方法，用于增强大型语言模型（LLM）。它通过从外部模型检索数据，并将检索到的相关数据作为上下文添加到LLM的问题或提示中，来回答关于自己数据的问题，或者在LLM训练时不公开的数据问题。

RAG的优点是能够保持企业的敏感数据在本地，并且在不需要对模型进行特定角色的微调的情况下，从通用模型中获得更相关的答案，大大减少了模型在回答生成中的幻觉现象。

**开发RAG支持的Llama 2聊天机器人的方法**：
最简单的方法是使用如LangChain和LlamaIndex这样的框架。这两个开源框架都提供了实现Llama 2的RAG功能的方便API，包括：

- 加载和分割文档
- 嵌入和存储文档分割
- 根据用户查询检索相关上下文
- 调用Llama 2进行查询和上下文生成答案

LangChain是一个更通用、更灵活的用于开发带有RAG能力的LLM应用程序的框架，而LlamaIndex作为数据框架，专注于将自定义数据源连接到LLM。两者的整合可能提供构建实际RAG应用程序的最佳性能和有效解决方案。在这个示例中，为了简单起见，我们将仅使用LangChain，搭配本地存储的PDF数据。

**安装依赖**：
对于这个演示，我们将使用Gradio来构建聊天机器人的UI，使用文本生成推理框架进行模型服务。
对于向量存储和相似性搜索，我们将使用FAISS。

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 数据处理

In [3]:
# 数据处理

DATA_PATH = "../data/"
DB_FAISS_PATH = "../data/vectorstore/db_faiss"

In [4]:
# 使用PyPDFDirectoryLoader加载整个目录 (还可以使用 PyPDFLoader 加载单个文件)

loader = PyPDFDirectoryLoader(DATA_PATH)

documents = loader.load()

Multiple definitions in dictionary at byte 0x2bb99 for key /MediaBox


In [5]:
print(len(documents))

4


In [6]:
print(documents[0].page_content[0:100])

书书书【
临床论著
】
老年人糖脂代谢病的中医证候要素调查
*
王舒沛
1
沈天然
1
，
2
王之浩
1
陈青松
1
，
2
△
摘要
:
目的
探讨老年人糖脂代谢病的中医病位和病性证候要素分布规律


In [7]:
# 将加载的文档分割成更小的块
# RecursiveCharacterTextSplitter 是一种常见的拆分器，它将长文本片段拆分为较小的、具有语义意义的块。 
# 其他splitters包括：
# SpacyTextSplitter
# NLTKTextSplitter
# SentenceTransformersTokenTextSplitter
# CharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

splits = text_splitter.split_documents(documents)

print(len(splits), splits[0])

112 page_content='书书书【\n临床论著\n】\n老年人糖脂代谢病的中医证候要素调查\n*\n王舒沛\n1\n沈天然\n1\n，\n2\n王之浩\n1\n陈青松\n1\n，\n2\n△\n摘要\n:\n目的\n探讨老年人糖脂代谢病的中医病位和病性证候要素分布规律' metadata={'source': '../data/老年人糖脂代谢病的中医证候要素调查_王舒沛.pdf', 'page': 0}


In [8]:
splits

[Document(page_content='书书书【\n临床论著\n】\n老年人糖脂代谢病的中医证候要素调查\n*\n王舒沛\n1\n沈天然\n1\n，\n2\n王之浩\n1\n陈青松\n1\n，\n2\n△\n摘要\n:\n目的\n探讨老年人糖脂代谢病的中医病位和病性证候要素分布规律', metadata={'source': '../data/老年人糖脂代谢病的中医证候要素调查_王舒沛.pdf', 'page': 0}),
 Document(page_content='。\n方法\n收集\n60\n岁及以上的糖脂代谢病患者\n3459\n例\n，\n对证候要素进行频数分析和聚类分析\n。\n结果\n3459\n例糖脂代谢病老年人群的病位证素以肾\n、\n肝\n、\n脾多见\n。\n病性证素分布前\n5', metadata={'source': '../data/老年人糖脂代谢病的中医证候要素调查_王舒沛.pdf', 'page': 0}),
 Document(page_content='病性证素分布前\n5\n位\n依次为\n:\n血虚\n、\n痰\n、\n阴虚\n、\n热\n、\n湿\n。\n糖脂代谢病中医证候要素组合规律多样\n，\n病位证素\n2\n种以上的组合形式占比\n21.8%\n(\n753/3459\n) ，', metadata={'source': '../data/老年人糖脂代谢病的中医证候要素调查_王舒沛.pdf', 'page': 0}),
 Document(page_content=') ，\n病\n性证候要素\n2\n种以上的组合形式占比\n64.0%\n(\n2215/3459\n)\n。\n聚类分析发现\n4\n种证候要素组合规律\n:\n气滞血瘀\n、\n肝阳上亢\n、\n脾阳气虚\n;\n痰湿证\n;\n热证\n;', metadata={'source': '../data/老年人糖脂代谢病的中医证候要素调查_王舒沛.pdf', 'page': 0}),
 Document(page_content=';\n热证\n;\n肾阴虚证与血虚证\n。\n结论\n老年人群糖脂代谢病的常见病位证素是肾\n，\n涉及肝\n、\n脾\n、\n心和胃\n，\n常见的病性证素是血虚\n、\n痰\n、\n阴虚\n、\n热\n、\n

In [9]:
# 从 HuggingFace 加载 Embedding 模型

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",
                                   model_kwargs = {"device": "cpu"})

/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# 将embedding存入向量数据库
db = FAISS.from_documents(splits, embedding=embeddings)

# 保存到本地
db.save_local(DB_FAISS_PATH)

# 2. 构建 Chatbot UI

In [11]:
import langchain
from queue import Queue
from typing import Any
from langchain.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import LLMResult
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from anyio.from_thread import start_blocking_portal

langchain.debug = True

# 向量数据文件路径
DB_FAISS_PATH = "../data/vectorstore/db_faiss"

# LLaMA-2 7B host port
LLAMA2_7B_HOSTPATH = "http://localhost:9909/v1"

model_dict = {
    "7b-chat" : LLAMA2_7B_HOSTPATH,
}

# 系统提示
system_message = {"role" : "system",
                  "content" : "You are a helpful assistant."}

In [12]:
# 加载embedding

embeddings = HuggingFaceEmbeddings(model_name = "/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/sentence-transformers/all-MiniLM-L6-v2",
                                   model_kwargs = {"device" : "cpu"})

In [13]:
# 加载 db

db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)

In [14]:
# 创建一个模型服务实例
# API : https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/openai.py#L144

llm = VLLMOpenAI(
    openai_api_key = "EMPTY",
    openai_api_base = LLAMA2_7B_HOSTPATH,
    model_name = "/slurm/home/yrd/shaolab/daiyizheng/resources/hf_weights/Qwen/Qwen1.5-7B",
    max_tokens = 300, 
    streaming=True
)

在构建基于检索的问答（RetrievalQA）链时，需要定义检索器（retriever）和模板。这两个组件在LangChain中扮演关键角色，用于处理和格式化查询以及生成答案。

1. **定义检索器**：检索器的主要功能是在向量数据库中执行语义相似性搜索。当RetrievalQA被调用时，LangChain会根据用户的查询在向量数据库中进行搜索。这种搜索基于语义相似性，目的是找到与查询最相关的文档或数据片段。检索到的结果（上下文）随后被传递给Llama，以便对存储在向量数据库中的数据进行查询和生成答案。

2. **定义模板**：模板则定义了将要发送到Llama进行生成的问题及其上下文的格式。Llama 2对于特殊标记有特殊的处理格式。在某些情况下，服务框架可能已经处理了这些特殊格式。如果没有，就需要编写自定义模板来正确处理这些特殊标记。模板的设计至关重要，因为它直接影响Llama 2如何理解问题和上下文，并据此生成回答。


In [15]:
# 模板

template = """
[INST]利用以下内容回答问题。如果没有提供上下文，请像人工智能助手一样回答。
{context}
问题: {question} [/INST]
"""

In [16]:
# retriever 检索器

retriever = db.as_retriever(search_kwargs = {"k": 6})

In [17]:
# 定义 chain

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    chain_type_kwargs = {
        "prompt": PromptTemplate(
            template = template,
            input_variables = ["context", "question"],
        ),
    }
)

In [18]:
# 测试

result = qa_chain({"query": "老年人糖脂代谢病主要有哪些病症? "})

print(result)

/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "老年人糖脂代谢病主要有哪些病症? "
}
[chain/error] [1:chain:RetrievalQA] s] Chain run errored with error:
"AssertionError()Traceback (most recent call last):\n\n\n  File \"/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/langchain/chains/base.py\", line 153, in invoke\n    self._call(inputs, run_manager=run_manager)\n\n\n  File \"/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py\", line 141, in _call\n    docs = self._get_docs(question, run_manager=_run_manager)\n\n\n  File \"/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py\", line 221, in _get_docs\n    return self.retriever.get_relevant_documents(\n\n\n  File \"/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/langchain_core/retrievers.py\", line 2

AssertionError: 

In [ ]:
# 此callback处理程序会将流式LLM响应放入队列中，以便gradio UI动态渲染。

job_done = object()

class MyStream(StreamingStdOutCallbackHandler):
    def __init__(self, q) -> None:
        self.q = q # 一个队列（Queue）对象
        
    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        self.q.put(token) # 将新生成的token放入之前在构造函数中定义的队列self.q中
        
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        # 将job_done对象放入队列self.q。这作为一个信号，表明语言模型已完成其工作。
        self.q.put(job_done)

In [ ]:
# gradio UI blocks

import gradio as gr
# 使用Gradio的Blocks接口创建一个新的UI布局
with gr.Blocks() as demo:
    # 定义UI布局
    chatbot = gr.Chatbot(height=600)
    with gr.Row():
        with gr.Column(scale=1):
            with gr.Row():
                # 创建一个下拉菜单，用于选择聊天机器人的模型。
                model_selector = gr.Dropdown(
                    list(model_dict.keys()),
                    value = "7b-chat",
                    label = "Model",
                    info = "Select the model",
                    interactive = True,
                    scale = 1
                )
                # 创建一个数字输入框，用于设置生成文本的最大tokens数量
                max_new_tokens_selector = gr.Number(
                    value = 512,
                    precision = 0,
                    label = "Max new tokens",
                    info = "Adjust max_new_tokens",
                    interactive = True,
                    minimum = 1,
                    maximum = 1024,
                    scale = 1
                )
            
            with gr.Row():
                # 创建一个滑动条，用于调整生成文本的“温度”（创造性）,范围是 0 到 1。
                temperature_selector = gr.Slider(
                    value = 0.6,
                    label = "Temperature",
                    info = "Range 0-1. Controls the creativity of the generated text.",
                    interactive = True,
                    minimum = 0.01,
                    maximum = 1,
                    step = 0.01,
                    scale = 1
                )
                # 创建另一个滑动条，用于调整“Top_p”参数，控制核采样。范围是 0.01 到 0.99。
                top_p_selector = gr.Slider(
                    value=0.9, 
                    label="Top_p", 
                    info="Range 0-1. Nucleus sampling.",
                    interactive = True, 
                    minimum=0.01, 
                    maximum=0.99, 
                    step=0.01, 
                    scale=1
                )
        # 创建一个文本框，用于用户输入聊天机器人的提示语。
        with gr.Column(scale=2):
            # 用户输入区域
            user_prompt_message = gr.Textbox(placeholder="Please add user prompt here",
                                             label="User prompt")
            with gr.Row():
                # 创建一个按钮，用于清除聊天内容。
                clear = gr.Button("Clear Conversation", scale=2)
                #  创建一个提交按钮，用于发送用户输入的提示语给聊天机器人。
                submitBtn = gr.Button("Submit", scale=8)
                
    # 使用 Gradio 的 State 创建一个状态变量，用于跨多个交互保持数据。
    state = gr.State([])
    
    # 处理用户的消息
    def user(user_prompt_message, history):
        '''
        user_prompt_message 是用户输入的消息
        history 是之前的对话历史
        '''
        # 如果用户输入不为空
        if user_prompt_message != "":
            # 将用户消息添加到对话历史中
            return history + [[user_prompt_message, None]]
        else:
            # 添加一条提示消息到对话历史，表示用户输入不能是空的。
            return history + [["Invalid prompts - user prompt cannot be empty", None]]
        
        
    # 用于配置、发送提示、渲染生成等的聊天机器人逻辑
    def bot(model_selector,
            temperature_selector,
            top_p_selector,
            max_new_tokens_selector,
            user_prompt_message,
            history,
            message_history
           ):
        # 初始化机器人的消息为空字符串
        bot_message = ""
        # 将历史记录中最后一条消息的机器人回复部分设置为空字符串
        history[-1][1] = ""
        # 创建一个字典，代表用户的角色和内容
        dialog = [
            {"role": "user", "content": user_prompt_message},
        ]
        # 将刚创建的对话条目添加到消息历史中
        messages_history += dialog
        # 创建一个队列用于处理异步任务的输出
        q = Queue()
        # 更新新的llama超参数
        llm.openai_api_base = model_selector
        llm.temperature = temperature_selector
        llm.top_p = top_p_selector
        llm.max_tokens = max_new_tokens_selector
        
        # 定义一个异步函数来运行LLM任务
        async def task(prompt):
            # 运行LLM任务，并将输出通过回调函数MyStream发送到队列 q。
            ret = await qa_chain.run(prompt, callbacks=[MyStream(q)])
            return ret
        
        # 使用上下文管理器来处理异步任务
        with start_blocking_portal() as portal:
            # 立即开始执行异步任务task
            portal.start_task_soon(task, user_prompt_message)
            while True:
                # 持续从队列q中获取token
                next_token = q.get(True)
                # 检查是否接收到job_done信号，如果是，则将机器人的消息添加到消息历史中，
                # 并返回更新后的历史记录和消息历史。
                if next_token is job_done:
                    messages_history += [{"role": "assistant",
                                          "content": bot_message}]
                    return history, messages_history
                # 否则，将接收到的token添加到bot_message和history[-1][1]中
                bot_message += next_token
                history[-1][1] += next_token
                yield history, messages_history
                
        def init_history(messages_history):
            '''用于初始化消息历'''
            messages_history = []
            messages_history += [system_message]
            return messages_history
        
        def input_cleanup():
            '''用于清理输入，这里返回一个空字符串'''
            return ""
        
        # 当用户在文本框中输入内容并按下回车键时，这个方法会被触发。
        user_prompt_message.submit(
            user, # user函数
            [user_prompt_message, chatbot], # 输入参数包括user_prompt_message（用户输入的文本）和chatbot（聊天机器人组件）
            [chatbot], # 输出参数是chatbot，这意味着user函数的结果将更新聊天机器人组件的状态。
            queue = False, # 不将事件放入队列中异步处理，而是立即处理。
        ).then(
            # bot函数
            bot, 
            # 输入参数包括各种控件的值
            [model_selector, temperature_selector, top_p_selector, max_new_tokens_selector, user_prompt_message, chatbot, state],
            # 输出参数是chatbot和state，这意味着bot函数的结果将更新聊天机器人组件的显示内容和聊天历史状态。
            [chatbot, state]
        ).then(
            input_cleanup, # input_cleanup 函数
            [], # 没有输入参数
            [user_prompt_message], # 输出参数是user_prompt_message，用于清空用户输入文本框。
            queue = False, # 不将事件放入队列中异步处理，而是立即处理。
        )
        # 当用户点击提交按钮时，这个方法会被触发。
        submitBtn.click(
            user,
            [user_prompt_message, chatbot],
            [chatbot],
            queue = False # 不将事件放入队列中异步处理，而是立即处理。
        ).then(
            bot,
            [model_selector, temperature_selector, top_p_selector, max_new_tokens_selector, user_prompt_message, chatbot, state],
            [chatbot, state]
        ).then(
            input_cleanup,
            [],
            [user_prompt_message],
            queue = False, # 不将事件放入队列中异步处理，而是立即处理。
        )
        
        # 清除按钮触发
        # 第1步: 当用户点击清除按钮时，这个方法会被触发。它执行一个简单的匿名函数（lambda: None），不进行任何操作，但会触发后续的 success 回调。
        # 第2步: .success(init_history, [state], [state]): 成功执行点击事件后，调用 init_history 函数来初始化聊天历史。
        # 输入参数是 state，用于重置聊天历史。
        # 输出参数也是 state，更新后的聊天历史将反映在状态中。
        clear.click(lambda: None, None, chatbot, queue=False).success(init_history, [state], [state])

In [ ]:
demo.queue().launch(server_name="0.0.0.0", port=6006)